In [76]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt 
import sklearn.utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
from numpy import where
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [77]:
df = pd.read_csv('Data_for_UCI_named.csv')
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [78]:
df.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [79]:
df = sklearn.utils.shuffle(df)
df = df.reset_index(drop=True)
df.shape
df.stabf.value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [8]:
print(df.shape)
print(list(df.columns))

(10000, 14)
['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2', 'g3', 'g4', 'stab', 'stabf']


In [83]:
df_unstable = df[df.stabf=='unstable']
df_stable = df[df.stabf=='stable'].sample(350)
data_df = df_unstable.append(df_stable)

In [85]:
x = data_df.drop(columns=['stabf'])
y = data_df[ 'stabf']

In [86]:
df['stabf'].unique()

array(['unstable', 'stable'], dtype=object)

In [87]:
df_stable = df[df.stabf=='stable']
df_unstable = df[df.stabf=='unstable'].sample(350)
data_df = df_stable.append(df_unstable)

In [88]:
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop=True)
data_df.shape
data_df.stabf.value_counts()

stable      3620
unstable     350
Name: stabf, dtype: int64

In [ ]:
data_df = data_df.drop(columns=['p1', 'p2', 'p3', 'p4', 'g1', 'g2', 'g3', 'g4'])
x = data_df.drop(columns='stabf')
y = data_df['stabf']


In [92]:
print(x.shape)
print(y.shape)

(3970, 5)
(3970,)


In [93]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
y_train.value_counts()

stable      2528
unstable     251
Name: stabf, dtype: int64

In [94]:
x_train = pd.get_dummies(x_train, drop_first=True)
x_test = pd.get_dummies(x_test, drop_first=True)
x_train.shape

(2779, 5)

In [95]:
def score_data(y_test,pred_y, clf):
    print('\n\n'+clf)
    print("\n CONFUSION MATRIX \n", confusion_matrix(y_test,pred_y))
    print(classification_report(y_test,pred_y))
    print("Accuracy Score is: ", accuracy_score(y_test,pred_y))

In [96]:
LR = LogisticRegression()
LR.fit(x_train, y_train)
pred_y = LR.predict(x_test)
score_data(y_test, pred_y, 'Logistic Regression')



Logistic Regression

 CONFUSION MATRIX 
 [[1092    0]
 [  93    6]]
              precision    recall  f1-score   support

      stable       0.92      1.00      0.96      1092
    unstable       1.00      0.06      0.11        99

    accuracy                           0.92      1191
   macro avg       0.96      0.53      0.54      1191
weighted avg       0.93      0.92      0.89      1191

Accuracy Score is:  0.9219143576826196


In [97]:
x_train

,tau1,tau2,tau3,tau4,stab
1151,1.222725,6.311725,2.785187,6.679192,-0.012893
3265,7.762585,2.334484,9.656120,2.779687,-0.029926
601,7.247672,8.946477,0.965732,1.281542,-0.038859
3929,7.099400,3.454731,3.441862,4.455297,0.049325
98,4.085900,2.285851,0.592499,2.073874,-0.035409
...,...,...,...,...,...
835,6.917664,9.068520,0.682066,1.184691,-0.019789
3264,7.282214,8.942957,0.961731,1.795484,-0.038290
1653,0.895326,5.397509,8.554880,1.308974,-0.051465
2607,0.893141,6.717047,7.782020,8.062814,-0.017933


In [99]:
x_train.columns = ['x0','x1','x2','x3','x4']
x_test.columns = ['x0','x1','x2','x3','x4']

In [101]:
XG = xgb.XGBClassifier()
XG.fit(x_train, y_train)
pred_y = XG.predict(x_test)
score_data(y_test, pred_y, 'XGB Classifier')

C:\Users\David\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:42:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGB Classifier

 CONFUSION MATRIX 
 [[1092    0]
 [   0   99]]
              precision    recall  f1-score   support

      stable       1.00      1.00      1.00      1092
    unstable       1.00      1.00      1.00        99

    accuracy                           1.00      1191
   macro avg       1.00      1.00      1.00      1191
weighted avg       1.00      1.00      1.00      1191

Accuracy Score is:  1.0
